### Script for generating prediction CSV files for Dev and Test sets

**GA-Ensembling weights and individual model predictions already available**

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading CSV from link
def read_csv_from_link(url):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path,delimiter="\t",error_bad_lines=False, header=None)
    return df

In [3]:
# Loading All Data
# tamil_train = read_csv_from_link('https://drive.google.com/file/d/15auwrFAlq52JJ61u7eSfnhT9rZtI5sjk/view?usp=sharing')
# tamil_dev = read_csv_from_link('https://drive.google.com/file/d/1Jme-Oftjm7OgfMNLKQs1mO_cnsQmznRI/view?usp=sharing')
# malayalam_train = read_csv_from_link('https://drive.google.com/file/d/13JCCr-IjZK7uhbLXeufptr_AxvsKinVl/view?usp=sharing')
malayalam_dev = read_csv_from_link('https://drive.google.com/file/d/1J0msLpLoM6gmXkjC6DFeQ8CG_rrLvjnM/view?usp=sharing')
# kannada_train = read_csv_from_link('https://drive.google.com/file/d/1BFYF05rx-DK9Eb5hgoIgd6EcB8zOI-zu/view?usp=sharing')
# kannada_dev = read_csv_from_link('https://drive.google.com/file/d/1V077dMQvscqpUmcWTcFHqRa_vTy-bQ4H/view?usp=sharing')

In [44]:
# Test
tamil_test = read_csv_from_link('https://drive.google.com/file/d/10RHrqXvIKMdnvN_tVJa_FAm41zaeC8WN/view?usp=sharing')
malayalam_test = read_csv_from_link('https://drive.google.com/file/d/1zx1wCC9A-Pp80mzbqixb52WlWQQ7ATyJ/view?usp=sharing')
kannada_test = read_csv_from_link('https://drive.google.com/file/d/1Px2CvIkLP_xaNhz_fCofW-7GGBCnSYsa/view?usp=sharing')

b'Skipping line 778: expected 2 fields, saw 3\n'


In [56]:
split = 'test'

In [57]:
# Uncomment If making predictions for Dev-Set
# malayalam_test = malayalam_dev
# split = 'dev'

In [58]:
# Mal Preprocess
malayalam_test = malayalam_test.iloc[:, 0:2]
malayalam_test = malayalam_test.rename(columns={0: "text", 1: "GT_label"})

### Load Model

In [47]:
import torch

In [48]:
print("GPU Nos: {}".format(torch.cuda.device_count()))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

# Change Device - CPU/GPU-0/GPU-1
torch.cuda.set_device(0)
device = 'cuda'
device = device if torch.cuda.is_available() else 'cpu'

GPU Nos: 2
Tesla P100-PCIE-12GB
Tesla P100-PCIE-16GB


In [49]:
# Load Weights dictionary

In [9]:
import pickle
saved_model_filename = 'GA_v1_malayalam'

with open("../../dev_preds/weights_" + saved_model_filename + ".pickle", 'rb') as handle:
    mw = pickle.load(handle)

In [10]:
mw

{'XLMroberta_custom_pretrained_Malayalam.pth': (4.987400383529981,
  'xlm-roberta-base'),
 'Distilbert_m_base_cased_Malayalam_weighted.pth': (1.358223048938526,
  'distilbert-base-multilingual-cased'),
 'Indic_bert_Malayalam_weighted.pth': (2.1480728379176064,
  'ai4bharat/indic-bert'),
 'Distilbert_m_base_cased_Malayalam.pth': (2.7947970649265406,
  'distilbert-base-multilingual-cased'),
 'XLMroberta_custom_pretrained_Malayalam_weighted.pth': (4.206574786073908,
  'xlm-roberta-base'),
 'MURIL_cased_temp_Malayalam.pth': (0.6841248293693636,
  'simran-kh/muril-cased-temp'),
 'XLMroberta_large_Malayalam_weighted.pth': (3.8948383954170698,
  'xlm-roberta-large'),
 'Mbert_base_cased_Malayalam.pth': (0.44254299249817775,
  'bert-base-multilingual-cased'),
 'Mbert_base_cased_Malayalam_weighted.pth': (0.4904552048096972,
  'bert-base-multilingual-cased'),
 'XLMroberta_large_Malayalam.pth': (1.4920680613973296, 'xlm-roberta-large'),
 'Indic_bert_Malayalam.pth': (1.389161641030305, 'ai4bharat/i

In [13]:
all_probs = []
import os

In [10]:
label_mapping = {
    'Not_offensive': 0, 
    'not-malayalam': 1, 
    'Offensive_Targeted_Insult_Group': 2, 
    'Offensive_Untargetede': 3, 
    'Offensive_Targeted_Insult_Individual': 4
}

In [13]:
# Loading Model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
from tqdm.notebook import tqdm

for index in tqdm(mw.keys()):
    model_name = index
    pretrained_key = mw[index][1]
    try:
        test_preds = np.load("../../test_probs/"+model_name+".npy")
        all_probs.append(test_preds)
        continue
    except:
        pass

    if pretrained_key == 'distilbert-base-multilingual-cased':
        tokenizer = BertTokenizer.from_pretrained(pretrained_key)
        model = BertForSequenceClassification.from_pretrained(pretrained_key, num_labels=5)
    else:
        tokenizer = AutoTokenizer.from_pretrained(pretrained_key)
        model = AutoModelForSequenceClassification.from_pretrained(pretrained_key, num_labels=5)

    state_dict = torch.load(os.path.join('../../finetuned_models/', model_name))
    model.load_state_dict(state_dict)
    model.eval()

    # Collecting Text and Labels
    test_batch_sentences = list(malayalam_test['text'])

    # Convert to Tensor
    test_encodings = tokenizer(test_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt")

    # Dataset
    from torch.utils.data import Dataset
    class malayalam_Offensive_Dataset(Dataset):
        def __init__(self, encodings):
            self.encodings = encodings

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(1) # Label is irrelevant, since we need predictions
            return item

        def __len__(self):
            return len(self.encodings["input_ids"])

    # Defining Datasets
    test_dataset = malayalam_Offensive_Dataset(test_encodings)

    ### Run Models-1
    from torch.utils.data import DataLoader
    from tqdm.notebook import tqdm
    from sklearn.metrics import classification_report, f1_score

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    # Dataloaders
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    test_preds = []
    with torch.set_grad_enabled(False):
        for batch in tqdm(test_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            for logits in outputs[1].cpu().numpy():
                test_preds.append(np.exp(logits)/np.sum(np.exp(logits)))
    
    np.save("../../test_probs/"+model_name+".npy", test_preds)
    all_probs.append(test_preds)
    del model
    torch.cuda.empty_cache()

In [63]:
all_probs = []
if split == 'test':
    for k in mw.keys():
        all_probs.append(np.load("../../test_probs/"+k+".npy"))
else:
    for k in mw.keys():
        all_probs.append(np.load("../../model_prediction_probs/preds_"+k+".npy"))

In [70]:
mw

{'XLMroberta_custom_pretrained_Malayalam.pth': (4.987400383529981,
  'xlm-roberta-base'),
 'Distilbert_m_base_cased_Malayalam_weighted.pth': (1.358223048938526,
  'distilbert-base-multilingual-cased'),
 'Indic_bert_Malayalam_weighted.pth': (2.1480728379176064,
  'ai4bharat/indic-bert'),
 'Distilbert_m_base_cased_Malayalam.pth': (2.7947970649265406,
  'distilbert-base-multilingual-cased'),
 'XLMroberta_custom_pretrained_Malayalam_weighted.pth': (4.206574786073908,
  'xlm-roberta-base'),
 'MURIL_cased_temp_Malayalam.pth': (0.6841248293693636,
  'simran-kh/muril-cased-temp'),
 'XLMroberta_large_Malayalam_weighted.pth': (3.8948383954170698,
  'xlm-roberta-large'),
 'Mbert_base_cased_Malayalam.pth': (0.44254299249817775,
  'bert-base-multilingual-cased'),
 'Mbert_base_cased_Malayalam_weighted.pth': (0.4904552048096972,
  'bert-base-multilingual-cased'),
 'XLMroberta_large_Malayalam.pth': (1.4920680613973296, 'xlm-roberta-large'),
 'Indic_bert_Malayalam.pth': (1.389161641030305, 'ai4bharat/i

In [64]:
weights = [mw[index][0] for index in mw]

In [65]:
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [66]:
GA = False
if GA:
    X = weights
else:
    X = np.ones_like(weights)/len(weights)
all_dev_preds = np.array(all_probs)

softmax_w = np.exp(X)/np.sum(np.exp(X))
weighted_all_dev_preds = np.array([sw*dpreds for sw, dpreds in zip(softmax_w, all_dev_preds)])
weighted_dev_preds = np.sum(weighted_all_dev_preds, axis = 0)
final_dev_preds = np.argmax(weighted_dev_preds, axis = 1)

final_dev_pred_labels = [inverse_label_mapping[int(k)] for k in final_dev_preds]
saved_model_filename = 'test_GA_v1_malayalam'
malayalam_test['labels'] = final_dev_pred_labels

In [67]:
malayalam_test.index.name = 'id'

In [68]:
malayalam_test

,text,GT_label,labels
id,,,
0,Gopichettante BGM um mammookayum ishtapedunnav...,Not_offensive,Not_offensive
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,Not_offensive,Not_offensive
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,Not_offensive,Not_offensive
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,Not_offensive,Not_offensive
4,avan Varum ente Makan Madhura Raja....,Not_offensive,Not_offensive
...,...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,Not_offensive,Not_offensive
1995,Manju chechi undel hero vendaa. Chechi vere le...,Not_offensive,Not_offensive
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,Not_offensive,Not_offensive


In [69]:
from sklearn.metrics import classification_report

print(classification_report(malayalam_test['GT_label'], malayalam_test['labels']))

                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1779
     Offensive_Targeted_Insult_Group       0.82      0.69      0.75        13
Offensive_Targeted_Insult_Individual       0.92      0.50      0.65        24
               Offensive_Untargetede       0.93      0.65      0.76        20
                       not-malayalam       0.90      0.93      0.91       163

                            accuracy                           0.97      1999
                           macro avg       0.91      0.75      0.81      1999
                        weighted avg       0.97      0.97      0.97      1999



In [32]:
# Saving predictions
malayalam_test.to_csv("../../test_preds/" + saved_model_filename + ".tsv", sep = '\t')
np.save("../../test_probs/"+saved_model_filename+".npy", weighted_dev_preds)

#### Performance of each model separately

In [1]:
import numpy as np
import pickle
saved_model_filename = 'GA_v1_malayalam'

with open("../../dev_preds/weights_" + saved_model_filename + ".pickle", 'rb') as handle:
    mw = pickle.load(handle)

In [2]:
mw

{'XLMroberta_custom_pretrained_Malayalam.pth': (4.987400383529981,
  'xlm-roberta-base'),
 'Distilbert_m_base_cased_Malayalam_weighted.pth': (1.358223048938526,
  'distilbert-base-multilingual-cased'),
 'Indic_bert_Malayalam_weighted.pth': (2.1480728379176064,
  'ai4bharat/indic-bert'),
 'Distilbert_m_base_cased_Malayalam.pth': (2.7947970649265406,
  'distilbert-base-multilingual-cased'),
 'XLMroberta_custom_pretrained_Malayalam_weighted.pth': (4.206574786073908,
  'xlm-roberta-base'),
 'MURIL_cased_temp_Malayalam.pth': (0.6841248293693636,
  'simran-kh/muril-cased-temp'),
 'XLMroberta_large_Malayalam_weighted.pth': (3.8948383954170698,
  'xlm-roberta-large'),
 'Mbert_base_cased_Malayalam.pth': (0.44254299249817775,
  'bert-base-multilingual-cased'),
 'Mbert_base_cased_Malayalam_weighted.pth': (0.4904552048096972,
  'bert-base-multilingual-cased'),
 'XLMroberta_large_Malayalam.pth': (1.4920680613973296, 'xlm-roberta-large'),
 'Indic_bert_Malayalam.pth': (1.389161641030305, 'ai4bharat/i

In [7]:
all_model_names = mw.keys()

In [10]:
all_test_preds = []
for model_name in all_model_names:
    all_test_preds.append(np.load("../../test_probs/"+model_name+".npy"))

In [12]:
from sklearn.metrics import classification_report

In [19]:
label_mapping = {
    'Not_offensive': 0, 
    'not-malayalam': 1, 
    'Offensive_Targeted_Insult_Group': 2, 
    'Offensive_Untargetede': 3, 
    'Offensive_Targeted_Insult_Individual': 4
}

# Collecting Text and Labels
test_batch_sentences = list(malayalam_test['text'])
test_batch_labels =  [label_mapping[x] for x in malayalam_test['label']]

for test_preds, mn in zip(all_test_preds, all_model_names):
    final_test_preds = np.argmax(test_preds, axis = 1)
    y_true = test_batch_labels
    y_pred = final_test_preds
    target_names = label_mapping.keys()
    report = classification_report(y_true, y_pred, target_names=target_names)
    print(mn)
    print(report)

XLMroberta_custom_pretrained_Malayalam.pth
                                      precision    recall  f1-score   support

                       Not_offensive       0.98      0.99      0.98      1765
                       not-malayalam       0.91      0.93      0.92       157
     Offensive_Targeted_Insult_Group       0.88      0.65      0.75        23
               Offensive_Untargetede       0.91      0.72      0.81        29
Offensive_Targeted_Insult_Individual       0.80      0.59      0.68        27

                            accuracy                           0.97      2001
                           macro avg       0.90      0.78      0.83      2001
                        weighted avg       0.97      0.97      0.97      2001

Distilbert_m_base_cased_Malayalam_weighted.pth
                                      precision    recall  f1-score   support

                       Not_offensive       0.97      0.95      0.96      1765
                       not-malayalam       0.78 